## Much of code is credit to: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import re
import os
import sys
import numpy as np
import pandas as pd
from pprint import pprint
import pickle

import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

import spacy

import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
#gets a specified index and returns text data from dataframe 
def iterator(index):
    
    labels = ['ID','Name','Date','topicName','scrubbedtext']
    podKnow_Data = pd.DataFrame.from_records(results, columns = labels)
    
    #isolate scrubbed text values and convert to lowercase to avoid duplicates
    scrubbedData = str(podKnow_Data.iloc[index-1:index, 4].values).lower()
    
    #remove junk values
    scrubbedData = scrubbedData.replace("\"", "").replace(",", "").replace("\'",  "").splitlines()
            
    return scrubbedData

In [3]:
#lemmitizes words
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp("".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [4]:
#tokenizes words    
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [5]:
#outputs an HTML file with pyLDAvis result        
def printToHTML(ldaModel, corpus, id2word):
    
            lda_display = pyLDAvis.gensim.prepare(ldaModel, corpus, id2word, sort_topics=False)
            pyLDAvis.enable_notebook()
            
            htmlFileName = file.replace(".txt_scrubbed", ".html")
            pyLDAvis.save_html(lda_display, htmlFileName)

In [6]:
#creates bigrams, formats the data and produces a model as a result
def formatDataAndModel(finalBigrams):
            #remove junk values 
            data_words_bigrams = finalBigrams.replace("(", "").replace(")", "").replace("'", " ").replace("," ,"")
            
            data_words_bigrams = data_words_bigrams.split()
            finalBigrams = (list((data_words_bigrams)))
            
            
            #separate bigrams by '_' character and combine them into a single value 
            finalBigrams = [i+ '_' + j for i,j in zip(finalBigrams[::2], finalBigrams[1::2])]
                        
    
            data_lemmatized = lemmatization(finalBigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
            #maps IDs to words
            id2word = corpora.Dictionary(data_lemmatized)
    
            #simply receives lemmitized text
            texts = data_lemmatized
    
            #maps new lemmitized data to IDs
            corpus = [id2word.doc2bow(text) for text in texts]
    
    
            #builds model 
            lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           alpha= 'auto',
                                           eval_every=5,
                                           per_word_topics = True,
                                           passes=20)
    
            doc_lda = lda_model[corpus]
                        
            printToHTML(lda_model, corpus, id2word)

In [7]:
results = []
counter = 0
totalList = []


saveLocation = r'C:\Users\Frank Einstein\Podknow\data\transcripts\gcsst\scrubbed'
for folderName,subfolders,fileName in os.walk(r'C:\Users\Frank Einstein\Podknow\data\transcripts\gcsst\scrubbed'):
    
    
    try:   
          for file in fileName:
            if str(file.endswith(".txt_scrubbed")):
                f = open(os.path.join(folderName,file),'rb')
            data = pickle.load(f)
            
            value0, value1, value2,value3, *extraWords = file.split('_')
            value4 = data
            rows = (value0,value1,value2,value3, value4)
            results.append(rows)
            
            finalBigrams = ""
            
            counter = counter + 1
            
            nlp = spacy.load('en', disable=['parser', 'ner'])
            
            #get all scrubbed data from a given podcast index
            data = iterator(counter)
            
            data_words =  list(sent_to_words(data))

            tokens = nltk.wordpunct_tokenize(str(data))
            
            bigram_measures = nltk.collocations.BigramAssocMeasures()
    
            #this block removes junk characters and only accepts words 3 letters or longer
            finder = BigramCollocationFinder.from_words(tokens)
            finder.apply_word_filter(lambda w: len(w) < 3)
            
            #find top 200 best bigrams
            bigrams = list(sorted(finder.nbest(bigram_measures.likelihood_ratio, 200)))
            
            
            #convert bigrams to string for formatting
            for x in bigrams: 
                finalBigrams += str(x)
                
                
            formatDataAndModel(finalBigrams)
           
            
    except:
        print("error")
              
        

c:\users\frank einstein\appdata\local\programs\python\python37\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
c:\users\frank einstein\appdata\local\programs\python\python37\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
c:\users\frank einstein\appdata\local\programs\python\python37\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concaten

error
